In [13]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

In [177]:
school = 'University of California, Riverside' # change to your school's name

In [178]:
df = pd.read_csv(school + '.csv') 

# replace wrong locations(string 'connections' in them) to nulls
no_value = df.current_location.str.contains('connections') 
df.loc[no_value,'current_location'] = np.NAN

# remove some strings not identified by geocode (!you can add more depending on your data)
l = ['Metropolitan','Area', 'Township of', 'Region', 'Greater', 'Metroplex', 'District']
df['current_location'] = df['current_location'].str.replace('|'.join(l), '', regex=True)

geolocator = Nominatim(user_agent="course_project")

In [179]:
# generate a dictionary mapping unique locations to corresponding coordinates (3 mins)
coor = {}
unique_locs = df.current_location.unique()
for loc in unique_locs:
    location = geolocator.geocode(loc, timeout = 600)
    if pd.isna(location) or pd.isna(loc):
        coor[loc] = np.NAN
    else:
        coor[loc] = (location.latitude, location.longitude)

In [180]:
df['coordinate'] = df.apply(lambda row: coor[row.current_location], axis =1)
df

,current_location,current_work,previous_work,latest_Education,majro,search_school,coordinate
0,"Los Angeles, California, United States","Co-Founder at Black Yeti Beverage, Inc.",Chief Brand Officer & Neuromodulation Director...,2020–Present,Clinical Psychology,"University of California, Riverside","(34.0536909, -118.242766)"
1,"Newport Beach, California, United States",CEO at US Business Funding,Director of app only division / Small ticket E...,"University of California, Riverside",NaN,"University of California, Riverside","(33.6170092, -117.9294401)"
2,"San Francisco, California, United States","CEO at Empiric, Inc.",Sales at DAQRI,ArtCenter College of Design,Product Design,"University of California, Riverside","(37.7790262, -122.419906)"
3,"Allen, Texas, United States",Co-Founder at TitaniusBots,Customer Representative & Sales II at Bank of ...,"University of California, Riverside",Multi/Interdisciplinary Studies,"University of California, Riverside","(33.1031744, -96.6705503)"
4,"Coppell, Texas, United States",Chief Executive Officer at Kings III Emergency...,"President at SoloProtect, A Division of Kings ...",University of Califonia Riverside,Mathmatics,"University of California, Riverside","(32.9552598, -97.0155703)"
...,...,...,...,...,...,...,...
2493,NaN,Administrative Business Partner at Google,Training Coordinator II (Nelson Staffing) at G...,"University of California, Riverside",History,"University of California, Riverside",NaN
2494,United States,Director Patient Financial Services at San Ant...,Diector Business Services at Pomona Valley Hos...,University of Redlands,Organizational Leadership,"University of California, Riverside","(39.7837304, -100.445882)"
2495,"San Marino, California, United States",Lecturer in Environmental History at Caltech,Associate Research Professor of History at Cl...,"University of California, Riverside",History,"University of California, Riverside","(34.1215947, -118.1057394)"
2496,"Gilbert, Arizona, United States",Senior Manager Channel Enablement at Arctic Wolf,Global Channel Sales Enablement at Gigamon,Education (same page),Economics,"University of California, Riverside","(33.3527632, -111.7890373)"


In [181]:
#check a specific location
location = geolocator.geocode('Frankfurt Rhine', timeout = 600)
location

Location(Frankfurter Straße, Urdenbach, Stadtbezirk 9, Düsseldorf, Nordrhein-Westfalen, 40593, Deutschland, (51.1549523, 6.8854454, 0.0))

In [175]:
#check NAN values and revise current locations to be searchable (change/add more strings to remove in l)
df[(df.coordinate.isna()) & (df.current_location.notna())]

,current_location,current_work,previous_work,latest_Education,majro,search_school,coordinate
337,"Greenville-Spartanburg-Anderson, South Carolina","President/Owner, Financial Advisor, CFP®, CRPC...","Vice President, Financial Advisor, CFP®, CRPC®...",University of South Carolina - The Moore Schoo...,Brazil/Portuguese,"University of California, Riverside",NaN
548,Frankfurt Rhine-Main,Head of Banking and Partner Strategy and Execu...,Senior Adviser at Holland FinTech,"University of California, Riverside",German Language and Literature,"University of California, Riverside",NaN
796,"Greenville-Spartanburg-Anderson, South Carolina","President/Owner, Financial Advisor, CFP®, CRPC...","Vice President, Financial Advisor, CFP®, CRPC®...",University of South Carolina - The Moore Schoo...,Brazil/Portuguese,"University of California, Riverside",NaN
1001,Ezana is a startup entrepreneur currently prom...,CEO at Afri Kash,Service Management Officer at United Nations,"University of California, Riverside",Political Science,"University of California, Riverside",NaN
1013,Laila Mickelwait is the Founder and CEO of the...,Founder/CEO at Justice Defense Fund,Founder at New Reality International,University of Southern California,Public Diplomacy,"University of California, Riverside",NaN
1782,CEO at Cyphen Limited,CEO at Cyphen Limited,NaN,Peking University,国际经济,"University of California, Riverside",NaN
